<a href="https://colab.research.google.com/github/flavio-mota/introd-analise-dados/blob/main/Limpeza_e_prepara%C3%A7%C3%A3o_de_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:#336699">Limpeza e preparação de dados</span>
<hr style="border:2px solid #0077b9;">

<br/>

<div style="text-align: center;font-size: 90%;">
    Autores:<br/>
    Flávio Belizário da Silva Mota<br/>
    Melise Maria Veiga de Paula
    <br/>



## Objetivos gerais:


*   Apresentar alguns códigos, em python, que podem ser usados para limpeza e preparação de dados, como:
  *   Tratar dados ausentes
  *   Transformações de dados


## Configurando o ambiente

O Colab já vem configurado com muitas biblioteca úteis para o desenvolvimento de notebooks jupyter com python. Entre essas bibliotecas, temos a `pandas` e a `numpy`, que utilizaremos nessa aula.

No nosso caso, não precisaremos instalar essas bibliotecas, apenas importá-las. Já a biblioteca `fuzzywuzzy` precisará ser instalada. Ela é uma bibliotecaque  implementa algoritmos baseados na distância de Levenshtein para calcular a diferença entre strings.

In [ ]:
!pip install fuzzywuzzy python-Levenshtein

In [ ]:
# É um padrão da comunidade a importação das bibliotecas com um "apelido",
# para facilitar o uso posteriormente. Então a pandas será chamada por pd e
# a numpy por np ao longo do código.
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz

## Carregando os dados

Vamos agora carregar os dados que serão utilizados nessa aula. Eles estão no formato csv e foram disponibilizados em uma url.

Para isso, utilizaremos a função `read_csv` da `pandas`. Ao chamar essa função, vamos informar a url na qual está o arquivo, bem como o separador utilizado e também a codificação do arquivo.

Como arquivos csv podem vir de diversas fontes, essas fontes podem alterar a codificação do arquivo para que, por exemplo, possam ser usados caracteres especiais, formatos de datas específicos, etc.

In [ ]:
url = 'https://raw.githubusercontent.com/flavio-mota/introd-analise-dados/main/bolsistas-de-iniciacao-cientifica.csv'
df = pd.read_csv(url, sep=';', encoding='latin-1')

Uma vez carregado na estrutura `DataFrame`, podemos utilizar todos os recursos da biblioteca pandas para realizar operações com nossos dados. Vamos começar analisando os 5 primeiros registros do conjunto:

In [ ]:
df.head()

Vamos verificar as dimensões desse arquivo, ou seja, quantas linhas e quantas colunas temos:

In [ ]:
df.shape

Podemos verificar também quais os tipos de dados que a `pandas` interpretou ao carregar os dados:

In [ ]:
df.info()

O resultado retornado apresenta que 12 colunas armazenam dados do tipo <code>object</code>. O tipo <code>object</code> pela documentação da biblioteca pandas, representa um tipo de "objeto arbitrário". As cadeias de caracteres também são entendidas pela biblioteca como sendo desse tipo.

As 2 últimas colunas foram interpretadas como valores numéricos.

Além disso, já é possível perceber que nem todas as colunas tem todos os valores presentes.

## Dados ausentes

Dados ausentes são comuns em muitas aplicações de análise de dados. A `pandas` tenta deixar o trabalho com dados ausentes menos problemático.

A biblioteca referencia os dados ausentes como **NA**, do inglês Not Available (indísponivel). Esses dados NA podem ser dados realmente inexistentes ou dados que existem, mas não foram observados, por problemas com a coleta de dados, por exemplo.

Ao limpar os dados para análise, em geral é importante fazer a análise nos próprios dados ausentes a fim de identificar problemas em sua coleta ou possíveis distorções provocadas por dados ausentes.

Podemos verificar quantos dados ausentes temos por coluna usando o código abaixo:

In [ ]:
null_df = df.isnull().sum()
null_df



---


Existem algumas funções do pandas que podemos utilizar para tratar dados ausentes:

| **Função** | **Descrição**                                                                                                                                                         |
|------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `dropna()`   | Remove linhas ou colunas que contêm valores ausentes (NaN).<br>Você pode especificar um limite de quantos valores ausentes<br> são aceitáveis antes de uma linha ou coluna ser descartada. |
| `fillna()`   | Substitui os valores ausentes em um DataFrame por um valor específico <br>ou usando um método, como 'ffill' (forward fill) para preencher com o último<br> valor válido anterior ou 'bfill' (backward fill) para preencher com o próximo valor válido. |
| `isnull()`   | Retorna um objeto booleano do mesmo tamanho que o DataFrame,<br> onde cada elemento é `True` se o correspondente no DataFrame original<br> for um valor ausente (NaN ou None).  |
| `notnull()`  | Funciona como o oposto de `isnull()`: retorna `True` para cada elemento <br> que não é um valor ausente (NaN ou None).                                                        |


Vamos começar filtrando os dados ausentes. Para isso, vamos descartar as linhas nas quais a coluna Campus: [Itajubá] está em branco:

In [ ]:
df_limpo = df.copy()
df_limpo = df_limpo.dropna(subset=['Campus: [Itajubá]'])
null_df = df_limpo.isnull().sum()
null_df



---
Vamos considerar agora a coluna ValorBolsa. São 13 ocorrências de valores nulos. Nesse caso, não vamos descartar esses registros, mas sim substituir os valores ausentes por 0.


In [ ]:
df_limpo['ValorBolsa'] = df_limpo['ValorBolsa'].fillna(0)
null_df = df_limpo.isnull().sum()
null_df

Por hora, vamos apenas excluir os outros registros que tem ocorrência de valores ausentes:

In [ ]:
df_limpo.dropna(inplace=True)

---
Vamos agora trabalhar com alguns dados textuais.
Primeiramente, vamos selecionar algumas colunas e transformar o texto dessas colunas em maísculas. Primeiro contamos as ocorrências distintas de termos sem essa operação e depois de aplicada a operação, verificamos como estão os dados:



In [ ]:
df_limpo['Modalidade do Programa de Bolsa:'].value_counts()

In [ ]:
df_limpo['Modalidade do Programa de Bolsa:'] = df_limpo['Modalidade do Programa de Bolsa:'].str.upper()
df_limpo['Modalidade do Programa de Bolsa:'].value_counts()

In [ ]:
df_limpo['Instituto'].value_counts()

In [ ]:
df_limpo['Instituto'] = df_limpo['Instituto'].str.upper()
df_limpo['Instituto'].value_counts()

In [ ]:
df_limpo['Órgão de financiamento'] = df_limpo['Órgão de financiamento'].str.upper()
df_limpo['Órgão de financiamento'].value_counts()

---
Podemos verificar que mesmo colocando todas as letras em maíusculo, o campo **Órgão de financiamento** tem algumas inconsistências.

Existem valores que se parecem muito. Vamos nos certificar quais são as strings semelhantes a biblioteca para comparar strings `fuzzywuzzy`. Primeiro, definimos uma função que será responsável por comparar as strings:

In [ ]:
def encontrar_similares(nome, lista, limite=70):
    # 'limite' define o score mínimo para considerar uma correspondência
    similares = process.extractBests(nome, lista, scorer=fuzz.token_sort_ratio, score_cutoff=limite)
    return similares

Vamos agora extrair os nomes únicos que existem no campo **Órgão de financiamento**:

In [ ]:
nomes_unicos = df_limpo['Órgão de financiamento'].unique()

Agora, para cada nome único, vamos encontrar os similares e associar uma pontuação de similaridade. Depois, são apresentados os resultados dos nomes e qual a similaridade dos outros com ele:

In [ ]:
# Dicionário para guardar as sugestões
sugestoes = {}

for nome in nomes_unicos:
    # Encontra nomes similares com um score de similaridade acima de 80
    resultados = encontrar_similares(nome, nomes_unicos)
    if len(resultados) > 1:
        # Guarda as sugestões apenas se existirem múltiplos itens similares
        sugestoes[nome] = resultados

# Exibir as sugestões
for nome, similares in sugestoes.items():
    print(f'Original: {nome}')
    for similar, score in similares:
        print(f'  Similar: {similar} (Score: {score})')
    print('\n')

Podemos alterar as dferentes ocorrências de FAPEMIG e CNPq:

In [ ]:
lista_substituicoes = {'CMPQ':'CNPQ',
                       'UniAO': 'UNIÃO',
                       'FAPEMING': 'FAPEMIG'
                       }
df_limpo['Órgão de financiamento'] = df_limpo['Órgão de financiamento'].replace(lista_substituicoes)
df_limpo['Órgão de financiamento'].value_counts()

---
Vamos analisar agora o campo `Implementação`.


In [ ]:
df_limpo['Implementação'].value_counts()

Para esse campo, iremos fazer uma transformação para que os valores sejam datas:

In [ ]:
df_limpo['Implementação'] = pd.to_datetime(df_limpo['Implementação'], format='%d/%m/%Y')
df_limpo['Implementação'].value_counts()

Agora que o formato do dado é um formato próprio para datas, podemos realizar algumas operações no campo com mais facilidade, como extrair o mês:

In [ ]:
df_limpo['mes_implementacao'] = df_limpo['Implementação'].dt.month
df_limpo['mes_implementacao'].value_counts()

---
Por fim, vamos dividir as informações que estão no campo **Vigência**:


In [ ]:
df_limpo['Vigência'].value_counts()

Serão criados dois novos campos: inicio e fim, com as devidas datas. Depois os campos serão convertidos para o tipo data:

In [ ]:
df_limpo[['inicio', 'fim']] = df_limpo['Vigência'].str.split(' a ', expand=True)
df_limpo['inicio'] = pd.to_datetime(df_limpo['inicio'], format='%m/%Y')
df_limpo['fim'] = pd.to_datetime(df_limpo['fim'], format='%m/%Y')
print(df_limpo['inicio'].value_counts())
print(df_limpo['fim'].value_counts())

Podemos agora criar um campo chamado duração baseado na diferença entre o início e fim da vigência:


In [ ]:
df_limpo['duração_dias'] = (df_limpo['fim'] - df_limpo['inicio']).dt.days

df_limpo['duração_meses'] = df_limpo['duração_dias'] // 30

print(df_limpo['duração_dias'].value_counts())
print(df_limpo['duração_meses'].value_counts())

Por fim, podemos salvar as aterações que fizemos no csv em um novo csv:

In [ ]:
df_limpo.to_csv('bolsistas-de-iniciacao-cientifica-limpo.csv',
                sep=';',
                index=False,
                encoding='latin-1')